## Guitar amplifier emulation research

In [ ]:
import numpy as np
import seaborn as sns
import keras
import keras.backend as K
import matplotlib.pyplot as plt
import librosa
import tensorflow as tf
import IPython

Verify GPU support

In [ ]:
tf.config.list_physical_devices()

Defining required constants

In [ ]:
INPUT_PATH = "data/float32/clean.wav"
OUTPUT_PATH = "data/float32/amp.wav"

TEST_INPUT_PATH = "data/float32/test_in.wav"
TEST_OUTPUT_PATH = "data/float32/test_out.wav"

TRAIN_PER_EPOCH = 1 # seconds
VAL_DURATION = 1 # seconds

BATCH_SIZE = 100
INFERENCE_BATCH_SIZE = 44

INPUT_SIZE = 441
MODEL_NAME = input()

if MODEL_NAME == None:
  raise ValueError("Enter model name")

Creating dataset from audio

In [ ]:
from dataset import RawWaveformLoader

train, valid = RawWaveformLoader.DataSplitter(input_path=INPUT_PATH, 
                                              output_path=OUTPUT_PATH, 
                                              train_per_epoch=TRAIN_PER_EPOCH, 
                                              valid_time=VAL_DURATION, 
                                              batch_size=BATCH_SIZE,
                                              input_size=INPUT_SIZE,
                                              dynamic_train=True).split()

demo = RawWaveformLoader(input_path=TEST_INPUT_PATH, 
                         output_path=TEST_OUTPUT_PATH, 
                         batch_size=INFERENCE_BATCH_SIZE,
                         input_size=INPUT_SIZE, 
                         interval_duration=10, 
                         start_offset=86,
                         end_offset=96)

In [ ]:
import keras.backend as K
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Add, BatchNormalization

from metrics import error_to_signal

LEARNING_RATE = 0.001
HIDDEN_UNITS = 64

K.clear_session()
input_layer = Input(batch_shape=(BATCH_SIZE, INPUT_SIZE, 1))
rnn = LSTM(HIDDEN_UNITS, return_sequences=True)(input_layer)
rnn = BatchNormalization()(rnn)
rnn = Dense(1)(rnn)
added = Add()([input_layer, rnn])

model = Model(inputs=[input_layer], outputs=added)

model.summary()

model.compile(optimizer=Adam(LEARNING_RATE, clipnorm=10), loss=error_to_signal, metrics=['mse', error_to_signal, 'mae'])

Let's define callbacks

In [ ]:
from keras.callbacks import BackupAndRestore, CSVLogger, TensorBoard

from callbacks import CheckpointAndSave

class PrintWeights(keras.callbacks.Callback):
  
  def __init__(self) -> None:
    super().__init__()
    self.last_loss = None
    
FOLDER = "artifacts"
subfolder = f"{FOLDER}/{MODEL_NAME}"

callbacks = [
  BackupAndRestore(f'{subfolder}/backup'),
  # CheckpointAndSave(f'{subfolder}/mse_checkpoint/', demo, model, BATCH_SIZE, save_best_only=True, save_weights_only=True, monitor='val_mse', verbose=1),
  # CheckpointAndSave(f'{subfolder}/ets_checkpoint/', demo, model, BATCH_SIZE, save_best_only=True, save_weights_only=True, monitor='val_error_to_signal', verbose=1),
  # CheckpointAndSave(f'{subfolder}/mae_checkpoint/', demo, model, BATCH_SIZE, save_best_only=True, save_weights_only=True, monitor='val_mae', verbose=1),
  # ReduceLROnPlateau(patience=10, factor=0.5, cooldown=10, monitor='loss', min_lr=1e-5),
  TensorBoard(log_dir=f'{subfolder}/logs'),
  CSVLogger(f'{subfolder}/logs.csv', append=True),
  PrintWeights()
]

In [ ]:
model.fit(train, batch_size=BATCH_SIZE, shuffle=True, epochs=10000, callbacks=callbacks)

In [ ]:
pred = model.predict(demo, batch_size=BATCH_SIZE)

In [ ]:
from scipy.io import wavfile

wavfile.write('demo.wav', 44100, pred.reshape(-1, 1))